In [35]:
from huggingface_hub import login
login()

In [4]:
!pip install  sentencepiece datasets peft  evaluate  -U accelerate -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 44.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uni

In [5]:
from datasets import  load_dataset, Dataset, DatasetDict, concatenate_datasets

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd

2024-02-13 17:17:41.875490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 17:17:41.875608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 17:17:42.008970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
data = pd.read_json("/kaggle/input/reviews/reviews.json")
data.text = data.text.astype(str)
data = data.sample(frac=1).reset_index(drop=True) ## 73665 rows


fifty = int(len(data) * 0.7)
data = data[:fifty] 


id2c = {0: 'neg', 1:'neutral', 2: 'pos'}
c2id = {"neg": 0, "neutral": 1, "pos": 2}
data['sentiment'].unique()

def get_features(row):
    y = 0 if row['sentiment'] == 'neg' else 1 if row['sentiment'] == 'neutral' else 2
    return {'text': row['text'], 'label': y}


size = int(len(data) * 0.2)
treino = data[size:]
teste = data[:size]


train_dict_list = treino.apply(get_features, axis=1).tolist()

trainDataset = Dataset.from_dict({"text": [item['text'] for item in train_dict_list], "label": [item['label'] for item in train_dict_list]})

validation_dict_list = teste.apply(get_features, axis=1).tolist()

validationDataset = Dataset.from_dict({"text": [item['text'] for item in validation_dict_list], "label": [item['label'] for item in validation_dict_list]})



In [7]:
dataset = DatasetDict({"train": trainDataset, "validation": validationDataset})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 41252
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 10313
    })
})

In [8]:
model_repo = "distilbert/distilbert-base-multilingual-cased"

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_repo, num_labels=3, id2label=id2c, label2id=c2id)
tokenizer = AutoTokenizer.from_pretrained(model_repo, add_prefix_space=True)

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [10]:
def tokenize_items(examples):
  text = examples["text"]

  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
      text,
      return_tensors="pt",
      truncation=True,
      padding=True,
      max_length=512
  )

  # Convertendo tensores para matrizes numpy
  numpy_tokenized_inputs = {key: value.numpy() for key, value in tokenized_inputs.items()}

  return numpy_tokenized_inputs

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  model.resize_token_embeddings(len(tokenizer))
tokenized_dataset = dataset.map(tokenize_items, batched=True)
tokenized_dataset

Map:   0%|          | 0/41252 [00:00<?, ? examples/s]

Map:   0%|          | 0/10313 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 41252
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10313
    })
})

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
from sklearn.metrics import f1_score
accuracy = evaluate.load('accuracy')

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)
  f1 = f1_score(predictions, labels, average='macro')

  return {"accuracy": accuracy.compute(predictions=predictions, references=labels), "f1_score": f1}

In [13]:
## Untrained model predictions
text_list = ["Grande erro da rede globo hoje!!", "Gostei muito do jogo do corinthians hoje", "Que prova dificil"]


for text in text_list:
  inputs = tokenizer.encode(text, return_tensors="pt")
  logits = model(inputs).logits


  predictions = torch.argmax(logits)
  print(text + " - " + id2c[predictions.tolist()])

Grande erro da rede globo hoje!! - neg
Gostei muito do jogo do corinthians hoje - neg
Que prova dificil - neg


In [14]:
peft_config = LoraConfig(
                task_type="SEQ_CLS", ##sequence classification,
                r=4, ##intrinsic rank of trainable weight matrix
                lora_alpha=32, ##this is like learning rate
                lora_dropout=0.01, ## probality of dropout
                target_modules = ['q_lin'] ##we apply lora to query layer
              )

In [15]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 629,763 || all params: 135,956,742 || trainable%: 0.46320836373086965


In [16]:
#hyperparameters
lr =  1e-3
batch_size = 4
num_epochs = 5

training_args = TrainingArguments(
    output_dir = model_repo + "-lora-multilingual-text-classification",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [19]:
trainer = Trainer( 
    model = model, 
    args=training_args, 
    train_dataset=tokenized_dataset["train"], 
    eval_dataset=tokenized_dataset["validation"], 
    tokenizer=tokenizer, 
    data_collator=data_collator, ##this will dynamically pad examples in each batch 
    compute_metrics=compute_metrics 
)

    
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.378500,0.440187,{'accuracy': 0.8783089304760981},0.896805
2,0.362200,0.383574,{'accuracy': 0.8894599049743043},0.905927
3,0.341900,0.371197,{'accuracy': 0.8969262096383206},0.912401
4,0.364000,0.382034,{'accuracy': 0.8984776495685057},0.913733
5,0.390200,0.392245,{'accuracy': 0.8998351595074178},0.914643


Trainer is attempting to log a value of "{'accuracy': 0.8783089304760981}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Checkpoint destination directory distilbert/distilbert-base-multilingual-cased-lora-multilingual-text-classification/checkpoint-10313 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Trainer is attempting to log a value of "{'accuracy': 0.8894599049743043}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8969262096383206}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8984776495685057}" of type <cla

TrainOutput(global_step=51565, training_loss=0.3521191885038339, metrics={'train_runtime': 5181.3372, 'train_samples_per_second': 39.808, 'train_steps_per_second': 9.952, 'total_flos': 2.77222500919296e+16, 'train_loss': 0.3521191885038339, 'epoch': 5.0})

In [24]:
## Trained model predictionsd
text_list = ["Grande erro da rede globo hoje", "Gostei muito do jogo do corinthians hoje", "Que prova dificil"]

model.to("cpu")

for text in text_list:
  inputs1 = tokenizer.encode(text, return_tensors="pt")
  logits1 = model(inputs1).logits


  predictions1 = torch.argmax(logits1)
  str = text + " - " + id2c[predictions1.tolist()]
  str

In [31]:
inputs = tokenizer.encode("Nao gostei desse produto, pessima qualidade", return_tensors="pt")
logits = model(inputs).logits


predictions = torch.argmax(logits)
id2c[predictions.tolist()]

'neg'

In [36]:
model.save_pretrained("pt-trained-model")

peft_model = "igoramf/lora-pt-sentiment-analysis"

model.push_to_hub(
    peft_model, use_auth_token=True
)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:821: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/129 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/igoramf/lora-pt-sentiment-analysis/commit/a89ffdcf024d9b43e9ed39e9b8831f46d1b372ec', commit_message='Upload model', commit_description='', oid='a89ffdcf024d9b43e9ed39e9b8831f46d1b372ec', pr_url=None, pr_revision=None, pr_num=None)